# Runbook

## Goals

- Configure Container Services Manager
- Add a MySQL service offering
- Review the changes made to K8S
- Bind an app to the service

## Architecture Review

![Ref Arch](imgs/reference_architecture.png)

## Ops Man & CF CLI Login

Log into Ops Man and add the KSM product tile.

https://pcf.rome.cf-app.com/

Log into cf cli.

## KSM Tile Overview

### Assign AZs and Networks
### Kubernetes Configuration

**A base64 encoded PEM Cluster CA certificate**

In [ ]:
kubectl config view | grep -i -C 2 '    certificate-authority-data:'

**Service Account & Token**

In [ ]:
cat service-account.yml

In [ ]:
kubectl -n kube-system get sa ksm-admin

In [ ]:
kubectl -n kube-system get clusterrolebinding ksm-cluster-admin

secret_name=$(kubectl get serviceaccount ksm-admin --namespace=kube-system -o jsonpath='{.secrets[0].name}')

secret_val=$(kubectl --namespace=kube-system get secret $secret_name -o jsonpath='{.data.token}')

echo ${secret_val} | base64 --decode

### Storage Configuration

S3 compatible bucket is required

Access ID & Key are required with "list and upload" permissions

### Container Registry Configuration (Optional)

GCP Registry Username: _json_key

GCP Registry Password: *Full json file for the service account*

## Deploy Spring Music

[Spring Music Repo](https://github.com/cloudfoundry-samples/spring-music)

In [ ]:
git clone https://github.com/cloudfoundry-samples/spring-music.git

In [ ]:
./spring-music/gradlew -p ./spring-music clean assemble

In [ ]:
cf push spring-music -f ./spring-music/manifest.yml --hostname spring-music

## Deploy Service Offerings

[KSM Samples Repo](https://github.com/cf-platform-eng/ksm-sample)

In [ ]:
git clone https://github.com/cf-platform-eng/ksm-sample.git

### Set Environment Variables

In [ ]:
export KSM_TARGET=https://ksm.sys.rome.cf-app.com
export KSM_USER=ksm_api_admin
export KSM_INSECURE=true

In [ ]:
export KSM_PASSWORD=

### Add A Service Offering

In [ ]:
./ksm offer list

In [ ]:
./ksm offer save ksm-sample/ci-charts/ksm-mysql ksm-sample/ci-charts/mysql-1.3.0.tgz

In [ ]:
./ksm offer list

### Enable A Service

In [ ]:
cf marketplace

In [ ]:
cf service-access

In [ ]:
cf enable-service-access mysql

In [ ]:
cf marketplace

### Create & Bind A Service

In [ ]:
cf create-service mysql small spring-music-db

http://spring-music.apps.rome.cf-app.com/

In [ ]:
cf restart spring-music

In [ ]:
cf service spring-music-db

In [ ]:
cf bind-service spring-music spring-music-db

In [ ]:
cf restage spring-music

## Verify

### MySQL

In [ ]:
cf create-service-key spring-music-db spring-music-db-servicekey

In [ ]:
cf service-key spring-music-db spring-music-db-servicekey

mysql -u root -p[password] -h [host]

mysql> use my_db;

select * from album;

### Kubernetes

In [ ]:
kubectl get namespaces

In [ ]:
kubectl config set-context --current --namespace=

In [ ]:
kubectl get configmaps

In [ ]:
kubectl get secrets

In [ ]:
kubectl get all

## Cleanup

In [ ]:
cf delete-service-key spring-music-db spring-music-db-servicekey -f

In [ ]:
cf unbind-service spring-music spring-music-db

In [ ]:
cf delete-service spring-music-db -f

In [ ]:
cf delete spring-music -f

In [ ]:
./ksm offer delete mysql